# SIVA(シヴァ)AI競馬開発エンジニアによる「競馬で始める機械学習」ハンズオン

## 目標
    機械学習で競馬を予測するAIを作り、本日開催の競馬の予測を行う。

## 本日の流れ
    1- 必要ライブラリの読み込み
    2- データの読み込み
    3- 前処理
    4- 予測アルゴリズムの構築
    5- 精度検証
    6- 本日開催の競馬の予測



## 1- 必要ライブラリの読み込み

In [48]:
#データ解析ライブラリ
import pandas as pd
#
import numpy as np
#
from sklearn.model_selection import train_test_split
#
from sklearn.preprocessing import scale

from sklearn.ensemble import RandomForestClassifier
#
from keras.utils.np_utils import to_categorical
#
from keras.layers import Activation,Dense
#
from keras.models import Sequential,load_model
#
from keras.layers.normalization import BatchNormalization

## 2- データの読み込み
    2016年~2018年の3年分の過去データ(data_horse_race.csv)を読み込む

In [49]:
df = pd.read_csv('data/data_horse_race.csv')

/Users/yuta_miyawaki/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
for c in df.columns:
    print(c,':',df.iloc[0][c])

開催年 : 2016
開催日付 : 105
競馬場コード : 6
開催回次 : 1
開催日目 : 1
レース番号 : 1
枠番 : 1
馬番 : 1
血統番号 : 2013105621
馬名 : プロジェクション
性別 : 2
毛色コード : 3
馬齢 : 3
東西所属コード : 1
調教師コード : 353
斤量 : 540
ブリンカー仕様区分 : 0
騎手コード : 1131
騎手見習いコード : 0
馬体重 : 472
増減符号 : -
増減差 : 2
着順 : 16
走破タイム : 1161
単勝オッズ : 1905
単勝人気 : 14
着差 : 29
グレードコード : nan
競争種別コード : 12
重量種別コード : 3
競争条件コード : 703
距離 : 1200
芝ダート : d
回り方向 : R
コース区分 : nan
本賞金 : 50000
出走頭数 : 16
天気コード : 1
芝馬場 : 0
ダート馬場 : 1
開催年_1走前 : 2015.0
開催日付_1走前 : 1114.0
競馬場コード_1走前 : 3.0
馬番_1走前 : 11.0
斤量_1走前 : 530.0
ブリンカー仕様区分_1走前 : 0.0
騎手コード_1走前 : 1154.0
騎手名_1走前 : 松若風馬
馬体重_1走前 : 474.0
異常区分コード_1走前 : 0.0
着順_1走前 : 14.0
1コーナー順位_1走前 : 0.0
2コーナー順位_1走前 : 0.0
3コーナー順位_1走前 : 9.0
4コーナー順位_1走前 : 10.0
単勝オッズ_1走前 : 290.0
単勝人気_1走前 : 8.0
ラスト3Fタイム_1走前 : 400.0
着差_1走前 : 29.0
グレードコード_1走前 : nan
競争種別コード_1走前 : 11.0
重量種別コード_1走前 : 3.0
距離_1走前 : 1150.0
芝ダート_1走前 : d
回り方向_1走前 : R
競争条件コード_1走前 : 703.0
コース区分_1走前 : nan
本賞金_1走前 : 50000.0
出走頭数_1走前 : 16.0
天気コード_1走前 : 3.0
芝馬場_1走前 : 0.0
ダート馬場_1走前 : 2.0
開催年_2走前 : 2015.0
開催日付_2走前 : 913.0
競馬

## 3- 前処理
    ・不要ファクタの削除
    ・カテゴリカルデータのone-hot値化

### ラベルデータの作成
    今回ラベルは[3着以下,3着以上]と設定する

In [51]:
#着順データを抽出する
target_data = df['着順']

In [52]:
target_data

0         16
1         10
2         14
3          8
4          5
5         12
6          6
7         15
8          4
9          9
10         2
11         1
12         3
13         7
14        11
15        13
16        15
17        10
18         8
19        14
20         1
21         6
22         3
23         7
24         4
25        12
26         5
27        11
28        13
29         9
          ..
116411    10
116412     8
116413    16
116414    17
116415     6
116416     8
116417     7
116418     0
116419     3
116420     1
116421     2
116422     5
116423     4
116424     9
116425     2
116426     3
116427     6
116428     5
116429    13
116430     1
116431    10
116432     8
116433     4
116434    14
116435    11
116436     7
116437     9
116438    16
116439    12
116440    15
Name: 着順, Length: 116441, dtype: int64

In [53]:
#着順がN着以内であれば1,3着以上であれば0に変換

#分類分けの閾値を設定
limit_order = 3

target_data = target_data.apply(lambda x:1 if x <= limit_order else 0)

In [54]:
target_data

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        1
11        1
12        1
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        1
21        0
22        1
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
116411    0
116412    0
116413    0
116414    0
116415    0
116416    0
116417    0
116418    1
116419    1
116420    1
116421    1
116422    0
116423    0
116424    0
116425    1
116426    1
116427    0
116428    0
116429    0
116430    1
116431    0
116432    0
116433    0
116434    0
116435    0
116436    0
116437    0
116438    0
116439    0
116440    0
Name: 着順, Length: 116441, dtype: int64

### 不要カラムを削除

In [59]:
#不要カラムのリストを作成しまとめて削除する
drop_columns_list = []

#馬を特定する情報を削除
drop_columns_list.extend(['血統番号','馬名'])

#レース前にわからないデータを削除リストに追加
drop_columns_list.extend(['着順','走破タイム','着差'])

#カテゴリデータであり種類が多いため、取捨選択をよく考える必要がある
drop_columns_list.extend(['調教師コード','騎手コード'])

pre_col = ['騎手名','騎手コード']
for i in range(1,4):
    for c in pre_col:
        drop_columns_list.append(c + '_'+str(i)+'走前')

#直前発表データのため今回は不使用
drop_columns_list.extend(['天気コード','芝馬場','ダート馬場','馬体重','増減符号','増減差'])

df = df.drop(drop_columns_list,axis = 1)

In [60]:
df

,開催年,開催日付,競馬場コード,開催回次,開催日目,レース番号,枠番,馬番,性別,毛色コード,...,同騎手3着内回数,今回競馬場経験数,今回距離1着回数,今回距離3着内回数,今回斤量経験回数,今回グレード経験数,短距離経験回数,マイル経験回数,中距離経験回数,長距離経験回数
0,2016,105,6,1,1,1,1,1,2,3,...,0,1,0,0,1,2,1,1,0,2
1,2016,105,6,1,1,1,1,2,1,7,...,0,0,0,0,0,4,4,0,0,4
2,2016,105,6,1,1,1,2,3,1,3,...,1,1,0,1,0,1,1,0,0,1
3,2016,105,6,1,1,1,2,4,2,1,...,0,1,0,0,2,2,2,0,0,2
4,2016,105,6,1,1,1,3,5,1,1,...,2,1,0,1,0,4,3,1,0,4
5,2016,105,6,1,1,1,3,6,2,5,...,0,0,0,0,1,1,1,0,0,1
6,2016,105,6,1,1,1,4,7,1,4,...,0,1,0,0,0,2,2,0,0,2
7,2016,105,6,1,1,1,4,8,2,1,...,0,1,0,0,7,7,7,0,0,7
8,2016,105,6,1,1,1,5,9,2,3,...,0,0,0,1,2,3,3,0,0,3
9,2016,105,6,1,1,1,5,10,2,3,...,1,1,0,1,0,1,1,0,0,1


### カテゴリカル変数をone-hot値に変更する

In [61]:
#カテゴリカル変数のリストを作成しまとめて削除する
dummies_columns_list = []

dummies_columns_list.extend(['競馬場コード','性別','毛色コード','東西所属コード',
                        'ブリンカー仕様区分','グレードコード','競争種別コード','重量種別コード',
                        '競争条件コード','芝ダート','回り方向','コース区分'])

#過去3走分データのカテゴリカル変数
pre_col = ['競馬場コード', 'ブリンカー仕様区分','異常区分コード','グレードコード'
           ,'競争種別コード','重量種別コード','競争条件コード','芝ダート'
           ,'回り方向','コース区分','天気コード','芝馬場','ダート馬場',]
for i in range(1,4):
    for c in pre_col:
        dummies_columns_list.append(c + '_'+str(i)+'走前')

df = pd.get_dummies(data = df,columns = dummies_columns_list)

### 空値をカラムの中央値などで埋める

In [66]:
for c in df.columns:
    df[c] = df[c].fillna(df[c].median())

### 予測用データの作成

In [67]:
#numpy配列に変換
x = df.values
y = target_data.values

In [68]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [69]:
#訓練とテストにデータを分ける
data_train , data_test , target_train , target_test = train_test_split(x,y,test_size=0.2)

## 4- 予測アルゴリズムの構築

In [70]:
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(data_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

## 5- 精度検証

In [73]:
print('Test score: {}'.format(forest.score(data_test, target_test)))

Test score: 0.7999055348018378
